### CHapter7
### The Graph Abstract Data Type
#### The graph abstract data type (ADT) is defined as follows:

- Graph() creates a new, empty graph.
- addVertex(vert) adds an instance of Vertex to the graph.
- addEdge(fromVert, toVert) Adds a new, directed edge to the graph that connects two vertices.
- addEdge(fromVert, toVert, weight) Adds a new, weighted, directed edge to the graph that connects two vertices.
- getVertex(vertKey) finds the vertex in the graph named vertKey.
- getVertices() returns the list of all vertices in the graph.
  in returns True for a statement of the form vertex in graph, if the given vertex is in the graph, False otherwise.

In [119]:
import sys
class Vertex(object):
    def __init__(self,key):
        self.id = key 
        self.connectedTo = {}
        self.color = 'white'
        self.dist = sys.maxsize
        self.pred = None
        self.disc = 0
        self.fin = 0
    def addNeighbor(self,nbr,weight=0):
        self.connectedTo[nbr]=weight
    def setColor(self,color):
        self.color = color
    def setDistance(self,d):
        self.dist = d
    def setPred(self,p):
        self.pred = p
    def setDiscovery(self,dtime):
        self.disc = dtime
    def setFinish(self,ftime):
        self.fin = ftime
    def getFinish(self):
        return self.fin
    def getDiscovery(self):
        return self.disc
    def getPred(self):
        return self.pred
    def getDistance(self):
        return self.dist
    def getColor(self):
        return self.color    
    def getConnections(self):
        return self.connectedTo.keys()
    def getId(self):
        return self.id
    def getWeight(self):
        return self.connectedTo[nbr]
    def __str__(self):
        return str(self.id) + ":color " + self.color + ":disc " + str(self.disc) + ":fin " + str(self.fin) + ":dist " + str(self.dist) + ":pred \n\t[" + str(self.pred)+ "]\n"


              

    

In [120]:
class Graph(object):
    def __init__(self):
        self.vertList = {}
        self.numVertices = 0
    def addVertex(self,key):
        self.numVertices +=1
        newVertex = Vertex(key)
        self.vertList[key]=newVertex
        return newVertex
    def getVertex(self,n):
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None
    def __contains__(self,n):
        return n in self.vertList
    def addEdge(self,f,t,cost=0):
        if f not in self.vertList:
            nv = self.addVertex(f)
        if t not in self.vertList:
            nv = self.addVertex(t)
        self.vertList[f].addNeighbor(self.vertList[t],cost)
    def getVertics(self):
        return self.vertList.keys()
    def __iter__(self):
        return iter(self.vertList.values())
        

In [121]:
g = Graph()
for i in range(6):
    g.addVertex(i)
g.addEdge(0,1,5)
g.addEdge(0,5,2)
g.addEdge(1,2,4)
g.addEdge(2,3,9)
g.addEdge(3,4,7)
g.addEdge(3,5,3)
g.addEdge(4,0,1)
g.addEdge(5,4,8)
g.addEdge(5,2,1)

for v in g:
    for w in v.getConnections():
        print("( %s , %s )" % (v.getId(), w.getId()))

( 0 , 5 )
( 0 , 1 )
( 1 , 2 )
( 2 , 3 )
( 3 , 5 )
( 3 , 4 )
( 4 , 0 )
( 5 , 2 )
( 5 , 4 )


### World Ladder puzzle
- 每次改变一个单词的一个字母,从一个单词变成另一个单词.
- ForExample:
 - Fool --->SAGE:
     FOOL
     POOL
     POLL
     POLE
     PALE
     SALE
     SAGE
   
 找到通过最少改变使得Word1--->Word2的方案.


### Step1.Building the Word Ladder Graph
Our first problem is to figure out how to turn a large collection of words into a graph. What we would like is to have an edge from one word to another if the two words are only different by a single letter. If we can create such a graph, then any path from one word to another is a solution to the word ladder puzzle. Figure 1 shows a small graph of some words that solve the FOOL to SAGE word ladder problem. Notice that the graph is an undirected graph and that the edges are unweighted.

![](./wordgraph.png)
### 方法:

- 构造多个桶,不同的桶拥有不同的标签.
- 例如我们有一个标签的"pop_"的桶
- 使用"_"作为通配符,和桶中的所有单词进行比较,pope,pops都将和"pop_"匹配.每一次找到一个匹配的单词,就把
  这个单词放在pop_标签的桶中.
- 可以使用Python自带的字典实现这个功能.


In [122]:
#构造这个桶
#Litscape.com
'''Classic Literature
Word Finder Tools
Word Analysis	
…
Word Finder Tools, Operating On The Litscape Default Censored Word List
Start ListsEnd ListsLength ListsAnagram Lists
Litscape Default Censored Word List
4 letter word list in the Litscape.com default word list
Litscape Default Censored Word List:
2404 words that are 4 letters long'''

from collections import defaultdict


def buildGraph(wordFile):
    wordlist = []
    #d = defaultdict(list)
    d =defaultdict(list)
    g=Graph()
    wfile = open(wordFile,'r')
    for line in wfile:
        word = line[:-1]
        
        for i in range(len(word)):
            bucket = word[:i]+"_"+word[i+1:]
            
            d[bucket].append(word)
            
    for bucket in d.keys():
        for word1 in d[bucket]:
            for word2 in d[bucket]:
                if word1 != word2:
                    g.addEdge(word1,word2)
    return g
            
      


### Step2.Implementing Breadth First Search
- 从一个顶点S开始,开始的颜色是灰色,以表示这个点已经被探索过了,还有两个其它值包括:距离和上一个点.
  分别给予初始值:0,None.
- 将这个初始点放进一个队列当中.
- 然后开始按部就班地探索这个队列前面的点.
- 我们通过迭代邻近列表(桶)来搜索队列中的每一个新节点.
- 检查邻近列表中的每一个节点的颜色.如果是白色.表示这个顶点未被搜索.相应的会有四种情况发生.
- 新的未被搜索的顶点nbr是灰色的.
- nbr的上一个点的被设置为最近被搜索过的点.
- 到nbr的距离设置为currentVert+1
- nbr被添加到队列的末尾

In [128]:
import Queue

def bfs(g,start):
    start.setDistance(0)
    start.setPred(None)
    vertQueue = Queue.Queue()
    vertQueue.put(start)
    while (vertQueue.qsize()>0):
        currentVert=vertQueue.get()
        for nbr in currentVert.getConnections():
            if nbr.getColor()=='white':
                nbr.setColor('gray')
                nbr.setDistance(currentVert.getDistance()+1)
                nbr.setPred(currentVert)
                vertQueue.put(nbr)
        currentVert.setColor('black')
g = buildGraph('four-word.txt')    
start = g.getVertex('fool')
bfs(g,start)

In [137]:
def traverse(y):
    x = y
    while (x.getPred()):
        print(x.getId())
        x = x.getPred()
    print(x.getId())

traverse(g.getVertex('next'))

next
neat
beat
boat
boot
foot
fool
